<a href="https://colab.research.google.com/github/pranaya-mathur/Deep-Learning-Projects/blob/master/FAISS_on_Colab_7th_Sept_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab,preprocessing,elasticsearch,inference]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
  Obtaining dependency information for farm-haystack[colab,elasticsearch,inference,preprocessing] from https://files.pythonhosted.org/packages/a3/e3/8a7ebb563df04300c9e2387ac31e8ffda809b21f276faa838b32cf4a74a7/farm_haystack-1.20.0-py3-none-any.whl.metadata
  Obtaining dependency information for canals==0.7.0 from https://files.pythonhosted.org/packages/08/2f/62e9e455c4cf183ebf6672c47befabb6a36a37a8cdc86256a5632d6ace10/canals-0.7.0-py3-none-any.whl.metadata
  Obtaining dependency information for events from https://files.pythonhosted.org/packages/25/ed/e47dec0626edd468c84c04d97769e7ab4ea6457b7f54dcb3f72b17fcd876/Events-0.5-py3-none-any.whl.metadata
  Obtaining dependency information for posthog from https://files.pythonhosted.org/packages/a7/73/35758818228c70348be4c3c6

In [2]:
path = 'models/faiss'

import os

if not os.path.exists(path):
    os.makedirs(path)

In [ ]:
!pip install farm-haystack[faiss]

In [3]:
from haystack.document_stores import FAISSDocumentStore

# initialize FAISS
document_store = FAISSDocumentStore(
    faiss_index_factory_str='Flat',
    sql_url=f'sqlite:///{path}/squad_dev.db',
    return_embedding=True
)

In [4]:
import json

with open('dev.json', 'r') as f:
    squad = json.load(f)

In [5]:
squad_docs = []

for sample in squad:
    squad_docs.append({
        'content': sample['context']
    })


In [6]:
document_store.write_documents(squad_docs)

Writing Documents: 20000it [00:22, 892.56it/s]


In [7]:
from haystack import Document

In [10]:
# create list of contexts
contexts = [sample['context'] for sample in squad]

# remove duplicates
contexts = list(set(contexts))

# create list of Document objects
squad_docs = [Document(content=sample) for sample in contexts]

In [12]:
squad_docs[:2]

[<Document: {'content': 'When suffering from sleep deprivation, active immunizations may have a diminished effect and may result in lower antibody production, and a lower immune response, than would be noted in a well-rested individual. Additionally, proteins such as NFIL3, which have been shown to be closely intertwined with both T-cell differentiation and our circadian rhythms, can be affected through the disturbance of natural light and dark cycles through instances of sleep deprivation, shift work, etc. As a result, these disruptions can lead to an increase in chronic conditions such as heart disease, chronic pain, and asthma.', 'content_type': 'text', 'score': None, 'meta': {}, 'id_hash_keys': ['content'], 'embedding': None, 'id': 'f073833dc6538d023ffedbc3e501e3d6'}>,
 <Document: {'content': 'The historian Francis Aidan Gasquet wrote about the \'Great Pestilence\' in 1893 and suggested that "it would appear to be some form of the ordinary Eastern or bubonic plague". He was able to

In [13]:
document_store.delete_all_documents()

                1. delete_all_documents() method is deprecated, please use delete_documents method
                For more details, please refer to the issue: https://github.com/deepset-ai/haystack/issues/1045
                


In [14]:
document_store.write_documents(squad_docs)

Writing Documents: 10000it [00:01, 6325.33it/s]


In [16]:
from haystack.nodes import DensePassageRetriever

retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model='facebook/dpr-question_encoder-single-nq-base',
    passage_embedding_model='facebook/dpr-ctx_encoder-single-nq-base',
    use_gpu=True,
    embed_title=True
)

document_store.update_embeddings(retriever=retriever)

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Create embeddings:  99%|█████████▊| 1200/1216 [00:18<00:00, 77.07 Docs/s]
Documents Processed: 10000 docs [00:20, 476.97 docs/s]


In [17]:
document_store.save(f'{path}/squad_dev.faiss')

In [18]:
del document_store, retriever

In [24]:
document_store = FAISSDocumentStore.load(

    index_path= f'{path}/squad_dev.faiss',

    config_path= f'{path}/squad_dev.json'

)

In [25]:
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model='facebook/dpr-question_encoder-single-nq-base',
    passage_embedding_model='facebook/dpr-ctx_encoder-single-nq-base',
    use_gpu=True,
    embed_title=True
)

In [26]:
retriever.retrieve('What subject is most abstract?')

[<Document: {'content': "A Turing machine is a mathematical model of a general computing machine. It is a theoretical device that manipulates symbols contained on a strip of tape. Turing machines are not intended as a practical computing technology, but rather as a thought experiment representing a computing machine—anything from an advanced supercomputer to a mathematician with a pencil and paper. It is believed that if a problem can be solved by an algorithm, there exists a Turing machine that solves the problem. Indeed, this is the statement of the Church–Turing thesis. Furthermore, it is known that everything that can be computed on other models of computation known to us today, such as a RAM machine, Conway's Game of Life, cellular automata or any programming language can be computed on a Turing machine. Since Turing machines are easy to analyze mathematically, and are believed to be as powerful as any other model of computation, the Turing machine is the most commonly used model 